In [21]:
import numpy as np
X = np.random.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB, GaussianNB
clf =  MultinomialNB()
clf.fit(X, y)

test = X[0]
test[1] = 0
test = np.random.randint(5, size=(100))

print(clf.predict_proba(test))

[[  7.94771063e-06   9.90030296e-01   9.86937379e-03   5.95542608e-08
    7.37611133e-05   1.85614682e-05]]


In [1]:
### normcdf
import numpy as np
from scipy.special import erfc
import math

def normcdfM(x, m=None, s=None):
    if m==None and s==None:
        z = x
    elif s==None:
        z = x-m
    else:
        z = (x-m)/s
        
    return 0.5*erfc(-z/math.sqrt(2))
    

In [2]:
### normpdf

import numpy as np
import math

def normpdfM(x, m=0, s=1):    
    o = 1.0/math.sqrt(2.0*math.pi*(s**2))
    
    return o*math.exp((-((x-m)**2)/(2*(s**2)))[0])
    

In [3]:
##tmean

import numpy as np

def tmean(m, index_max, n_samps):
    
    K = m.shape[0]
    u = np.random.randn(n_samps,1)

    t = m[index_max]*np.ones((K, 1)) - m 
    tr = t
    t = np.delete(t, index_max, 0)
    
    s = np.tile(u, (1, K-1)) + np.tile(t,(1, n_samps)).conj().T
    z = np.mean(np.prod(safenormcdf(s.conj().T), 0))

    tm = np.zeros(m.shape[0])
    for r in range(0, K):
        sr = np.tile(u, (1, K)) + np.tile(tr, (1, n_samps)).conj().T
        sr.take([r, index_max], axis=1)
        
        nr = np.mean(safenormpdf(u.conj().T + m[index_max] 
                                 - m[r])*np.prod(safenormcdf(sr.conj().T), 0))
        
        if r == index_max:
            tm[r] = 0.0
        else:
            tm[r] = m[r] - nr/z
            
    tm[index_max] = np.sum(m, axis=0) - np.sum(tm, axis=0)
    tm = tm.conj().T                    
            
    return tm, z
                                                               

def safenormcdf(x):
    #x = x[0]
    thresh=-10;
    x[np.nonzero(x<thresh)] = thresh
    return normcdfM(x)

def safenormpdf(x):
    x=x[0]
    thresh=35;
    x[np.nonzero(x<-thresh)] = -thresh
    x[np.nonzero(x>thresh)] = thresh
    return normpdfM(x)

def safelogsc(x):
    if x<1e-300:
        x = 1e-200
    elif x>1e300:
        x = 1e300
    return math.log(x)

def safelog(x):
    x[np.nonzero(x<1e-300)] = 1e-200
    x[np.nonzero(x>1e300)] = 1e300
    return math.log(x)
        

In [5]:
def generate(n):

    u = 4*np.random.rand(n,2)-2    
    
    i = np.nonzero( (u[:,0]**2 + u[:,1]**2 > 0.1) & (u[:,0]**2 + u[:,1]**2 < 0.5) )
    j = np.nonzero( (u[:,0]**2 + u[:,1]**2 > .6) & (u[:,0]**2 + u[:,1]**2 < 1) )
    X = u[np.concatenate((i[0],j[0])),:]

    t = np.zeros((i[0].shape[0],1), dtype=int)
    t = np.concatenate((t, np.ones((j[0].shape[0],1), dtype=int)))
    x = 0.1*np.random.randn(i[0].shape[0],2)
    
    k = np.nonzero(x[:,0]**2 + x[:,1]**2 < 0.1)
    X = np.concatenate((X, x[k[0],:]))
    t = np.concatenate((t, 2*np.ones((k[0].shape[0],1), dtype=int)))
    X = np.concatenate((X, np.random.randn(X.shape[0],8)), 1)

    
    return X, t


In [44]:
import numpy as np
import time

def dist(X,Y,Z): # X, Y, Z matrices
    
    nx = X.shape[0]
    ny = Y.shape[0]
    
#     distance= (np.sum(np.dot((X**2), Z), 1)*np.ones(ny)+
#               np.ones((nx,1))*(np.sum(np.dot((Y**2), Z), 1).conj().T) 
#                - 2*(np.dot(np.dot(X, Z), Y.conj().T))) 

    distance= (np.sum(np.dot((X**2), Z), 1).reshape(nx,1)*np.ones(ny)+
              np.ones((nx,1))*(np.sum(np.dot((Y**2), Z), 1).conj().T) 
               - 2*(np.dot(np.dot(X, Z), Y.conj().T))) 
           
    return distance


def VarMultProbRegGP(X, t, X_test=0, t_test=0, theta=0, theta_estimate=0, 
                     nos_its=0, kernel_Type=0, poly_kernel_power=0, thresh=0):
    
    start = time.time()*1000
    SMALL_NOS = 1e-10

    nos_samps_tg = 1000

    nos_samps_is = 1000

    sigma = 1e-6

    tau = 1e-6

    C = np.unique(t).size
    N, D = X.shape
    Y = np.random.randn(N, C)
    M = np.random.rand(N, C)
    beta = theta
    Theta = np.diag(theta)

    psi = np.ones((1, theta.size)) 
    In = np.eye(N)
    Ic = np.eye(C)

    diff = 1e100
    its = 0

    K = np.exp(-dist(X,X,Theta)) + np.eye(N)*SMALL_NOS; # TODO: implement create kernel function
    
    iK = np.linalg.inv(K+In)
    Ki = np.dot(K, iK)
    
    #THETA = []
    LOWER_BOUND = [-1e-3]
    #PL = []
    constant = (-0.5*C*np.trace(np.dot(K, iK))
                -0.5*C*np.trace(iK)
                -0.5*C*safelogsc(abs(np.linalg.det(K)))
                +0.5*C*safelogsc(abs(np.linalg.det(np.dot(K, iK))))
                -0.5*N*C*safelogsc(2*math.pi)
                +0.5*N*C + 0.5*N*safelogsc(2*math.pi))
    
    test_err = []
    while its<nos_its and diff>thresh:
        its += 1
        
        for k in range(0, C):
            M[:,[k]] = np.dot(Ki, Y[:,[k]])
  
        #print ("started Y %d"%(time.time()*1000-start))
        lower_bound = 0    
        for n in range(0, N):
            a, z = tmean(M[[n],].conj().T, t[n], nos_samps_tg)
            Y[n] = a
            lower_bound = lower_bound + safelogsc(z)
            
        #print ("completed Y %d"%(time.time()*1000-start))
        
        if its == 2: 
             LOWER_BOUND[0] = LOWER_BOUND[1] #### WHY? 
        
#         lower_bound = (lower_bound-0.5*C*np.trace(np.dot(K, iK))
#                                   -0.5*np.sum(np.diag(np.dot(np.dot(M.conj().T, iK), M)))
#                                   -0.5*C*np.trace(iK)
#                                   -0.5*C*safelogsc(abs(np.linalg.det(K)))
#                                   +0.5*C*safelogsc(abs(np.linalg.det(np.dot(K, iK))))
#                                   -0.5*N*C*safelogsc(2*math.pi)
#                                   +0.5*N*C + 0.5*N*safelogsc(2*math.pi))

        lower_bound = (lower_bound + constant -0.5*np.sum(np.diag(np.dot(np.dot(M.conj().T, iK), M))))
        
        LOWER_BOUND.append([lower_bound]) 
        if its == 2: 
             LOWER_BOUND[0] = LOWER_BOUND[1]                                            
                                                    
        diff = abs(100*(lower_bound - LOWER_BOUND[-2])/LOWER_BOUND[-2])
        #print ("completed iteration %d"%(time.time()*1000-start))
        
    ####### predict ########

    n_test = X_test.shape[0]
    K_test = np.exp(-dist(X,X_test,Theta))
    K_test_self = np.exp(-dist(X_test,X_test,Theta))

    S = (np.diag(K_test_self) - np.diag(np.dot(np.dot(K_test.conj().T, iK), K_test))).conj().T

    predictive_likelihood = 0
    res = (np.dot(np.dot(Y.conj().T, iK), K_test)).conj().T
    P_test = np.ones((n_test, C))
    u = np.random.randn(nos_samps_tg, 1)

    for n in range(0, n_test):
        for i in range(0, C):
            pp = np.ones((nos_samps_tg, 1))
            for j in range(0, C):
                if j!=i:
                    pp = pp*safenormcdf(u+(res[n,i]-res[n,j])/(math.sqrt(1+S[n])))

            P_test[n,i] = np.mean(pp)
        P_test[n] = P_test[n]/np.sum(P_test[n])

        #predictive_likelihood = predictive_likelihood + safelog(P_test[n, t_test[n]])
        
    #print ("completed prediction %d"%(time.time()*1000-start))
    return P_test
        
    
# X_1 = np.random.rand(20, 5)
# X_2 = 100*np.random.rand(20,5)
# X = np.concatenate((X_1, X_2))
# X_test = np.random.rand(50,5)
# y = np.array([0 for x in range(0, 20)]+[1 for x in range(0, 20)])
# theta = np.random.rand(5)

# X = np.array([[0.0, 0.0], [1.0, 1.0],[2.0, 2.0]])
# t = np.array([0, 1, 2]) #starting from 0
# X_t = np.array([[0.0, 0.0], [1.0, 1.0], [2.0, 2.0]])
# theta = np.array([0.169, 0.334])

X, t = generate(500)
X_t, t_t = generate(5000)
theta = np.array([1.1348e+01, 8.8135e+00, 1.7372e-03, 2.2966e-03, 1.7476e-03, 
                  1.6263e-03, 1.8975e-03, 1.2296e-03,  2.4512e-03, 1.6161e-03])

prob = VarMultProbRegGP(X, t, theta=theta, nos_its=50, thresh = 0.1, X_test=X_t)    
pred = np.argmax(prob, axis=1)
pred = pred.reshape(pred.shape[0])
t_t = t_t.reshape(t_t.shape[0])

error = pred - t_t
print t_t.shape
print np.nonzero(error)[0].shape



(1080,)
(27,)


In [51]:
import numpy as np
import ast, dataIO, common_functions
from sklearn import cross_validation, linear_model, preprocessing, svm
reload(dataIO)
reload(common_functions)

postures = {"left_hand":["4", "8", "11"], "right_hand":["1", "7", "10"], 
            "index_finger":["3", "5", "12"], "two_hand":["2", "6", "9"]}

locations = []
bod = []
targets_x = []
targets_y = []
y = []
touch_centers = []

def learn_offset(points, targets):
    regr = linear_model.LinearRegression()
    regr.fit(points, targets)
    
    return regr

def process_twohand(userid, filenos, posture):
    
    centers = dataIO.get_key_centers()
    
    for fileno in filenos:
        filename = "/home/dimitar/Desktop/Python/experiment/results/"+str(userid)+"_"+fileno+"up.txt"
        with open(filename, "r") as f:
            lines = f.read().splitlines()
            lines = map(lambda x: x.split('\t'), lines[1:])
            for line in lines:
                letter = line[1]
                location = list(ast.literal_eval(line[3]))
                center = centers[letter]
                
                if not dataIO.iscorrect(location, center):
                    continue
                
                if line[0] == "left":
                    y.append(posture)
                else:
                    y.append(posture+1)
                
                touch_centers.append(center)
                targets_x.append(center[0]-location[0])
                targets_y.append(center[1]-location[1])                
                locations.append(location)
                bod.append(dataIO.createlist(line[-1]))
    

def process_posture(userid, filenos, posture): 
    
    centers = dataIO.get_key_centers()
    
    for fileno in filenos:
        filename = "/home/dimitar/Desktop/Python/experiment/results/"+str(userid)+"_"+fileno+"up.txt"
        with open(filename, "r") as f:
            lines = f.read().splitlines()
            lines = map(lambda x: x.split('\t'), lines[1:])
            for line in lines:
                letter = line[0]
                location = list(ast.literal_eval(line[2]))
                center = centers[letter]
                
                if not dataIO.iscorrect(location, center):
                    continue
                
                targets_x.append(center[0]-location[0])
                targets_y.append(center[1]-location[1])                
                touch_centers.append(center)
                locations.append(location)
                bod.append(dataIO.createlist(line[-1]))
                y.append(posture)
                

def read_data(userid):    
    keys = postures.keys()
    
    posture = 0
    for key in keys:
        filenos = postures[key]
        if key == "two_hand":
            process_twohand(userid, filenos, posture)
            posture += 2
        else:
            process_posture(userid, filenos, posture)
            posture += 1

read_data(8)

locations = np.array(locations)
bod = np.array(bod)
targets_x = np.array(targets_x)
targets_y = np.array(targets_y)
y = np.array(y)
touch_centers = np.array(touch_centers)

print("MSE on x before regression: %.2f"  % np.mean(targets_x ** 2))
print("MSE on y before regression: %.2f"  % np.mean(targets_y ** 2))
within_before = common_functions.circle_button_error(locations, touch_centers)

foldno = 1
se_x = []
se_y = []
within_after = []
kf = cross_validation.KFold(len(y), n_folds=5, shuffle=True)

for train_index, test_index in kf:    
    points_train, points_test = locations[train_index], locations[test_index]
    bod_train, bod_test = bod[train_index], bod[test_index]
    t_x_train, t_x_test = targets_x[train_index], targets_x[test_index]
    t_y_train, t_y_test = targets_y[train_index], targets_y[test_index]
    y_train, y_test = y[train_index], y[test_index]
    centers_train, centers_test = touch_centers[train_index], touch_centers[test_index]
    
    scaler = preprocessing.StandardScaler().fit(bod_train)  
    bod_scaled = scaler.transform(bod_train)
    test = scaler.transform(bod_test)
    
    theta = np.array([0.01 for x in range(0, 24)])
    prob = VarMultProbRegGP(bod_scaled, y_train, theta=theta, nos_its=50, thresh = 0.1, X_test=test)
    
    pred = np.argmax(prob, axis=1)
    pred_r = pred.reshape(pred.shape[0])

    error = pred_r - y_test
    print y_test.shape
    print np.nonzero(error)[0].shape
    
    #clf = svm.SVC(C=100, kernel='rbf', gamma=0.01, cache_size=500)
    #clf.fit(bod_scaled, y_train)
    
    regr_x = []
    regr_y = []
    
    for i in range(0,5):
        index = np.where(y_train==i)[0]
        regr_x.append(learn_offset(points_train[index], t_x_train[index]))
        regr_y.append(learn_offset(points_train[index], t_y_train[index]))
        
    new_points = []
    for i in range(len(points_test)):        
        point = points_test[i]
        
        pred_x = regr_x[pred[i]].predict(point)
        pred_y = regr_y[pred[i]].predict(point)
        
        new_points.append([point[0]+pred_x, point[1]+pred_y])
     
    within_after.append(common_functions.circle_button_error(new_points, centers_test))
    new_points = np.array(new_points).T
    centers_test = centers_test.T
    
    se_x.append((new_points[0]-centers_test[0])**2)
    se_y.append((new_points[1]-centers_test[1])**2)
    
#     foldno +=1    

se_x = np.array([item for sublist in se_x for item in sublist])
se_y = np.array([item for sublist in se_y for item in sublist])

print 
print("Mean MSE on x: %.2f" % np.mean(se_x))
print("Mean MSE on y: %.2f" % np.mean(se_y))      
within_after = np.mean(np.array(within_after), 0)

MSE on x before regression: 305.11
MSE on y before regression: 128.41
(156,)
(20,)
(156,)
(27,)
(156,)
(22,)
(155,)
(18,)
(155,)
(32,)

Mean MSE on x: 75.08
Mean MSE on y: 117.36
